In [1]:
from get_features import getLbp
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
print 'Loading Data....'
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
x_train = np.squeeze(x_train)
# print len(x_train)
# sys.exit()


# Parameters
learning_rate = 0.01
training_epochs = 20


# Network Parameters
n_hidden_1 = 128
n_hidden_2 = 50
n_input = 256

# tf Graph input
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}

# Building the encoder
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))

    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))

    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X

cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

init = tf.initialize_all_variables()


Loading Data....


In [3]:
Bsize = 256
Tsize = len(x_train)
Nbatch = int(Tsize/Bsize)
if Tsize/(Bsize*1.0) != Nbatch:
    Nbatch = Nbatch + 1
print 'Total Number of Batches:',Nbatch
nEpochs = 20

# sess = tf.Session()

#     _,c = sess.run([optimizer, cost],feed_dict={X: x_train[:256]})
#     print c
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(nEpochs):
        print "Epoch:",epoch+1,
        for i in range(Nbatch):
            if(i+1 == Nbatch):
                a = (i)*Bsize+1
                b = Tsize
            else:
                a = (i)*Bsize+1
                b = (i+1)*Bsize

    #             print a-1,b
            batch_x = x_train[a-1:b]
    #             print np.shape(x_train),np.shape(batch_x)
            _,c = sess.run([optimizer, cost], feed_dict={X: batch_x})
        print "cost:",c
    print("Optimization Finished!")
    
    w,b = sess.run([weights, biases])


Total Number of Batches: 47
Epoch: 1 cost: 250.34
Epoch: 2 cost: 250.082
Epoch: 3 cost: 249.912
Epoch: 4 cost: 249.808
Epoch: 5 cost: 249.752
Epoch: 6 cost: 249.74
Epoch: 7 cost: 249.73
Epoch: 8 cost: 249.729
Epoch: 9 cost: 249.718
Epoch: 10 cost: 249.717
Epoch: 11 cost: 249.714
Epoch: 12 cost: 249.713
Epoch: 13 cost: 249.713
Epoch: 14 cost: 249.711
Epoch: 15 cost: 249.709
Epoch: 16 cost: 249.708
Epoch: 17 cost: 249.708
Epoch: 18 cost: 249.7
Epoch: 19 cost: 249.655
Epoch: 20 cost: 249.654
Optimization Finished!


In [18]:
np.save('ae-weights.npy',w)
np.save('ae-biases.npy',b)

dict